# Exam #

By Nina Frandsen Jensen & Mathilde Maix

**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Optimal taxation with government consumption](#toc1_)    
- 2. [Problem 2: Labor adjustment costs](#toc2_)    
- 3. [Problem 3: Global optimizer with refined multi-start](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [5]:
# Imports 
import math 
import scipy.optimize
import numpy as np
import pandas as pd
import sympy as sm
from scipy import optimize
from tabulate import tabulate
from scipy import optimize
from sympy import symbols, lambdify
from scipy.optimize import minimize_scalar
from matplotlib import pyplot as plt

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

import Exam

## 1. <a id='toc1_'></a>[Problem 1: Optimal taxation with government consumption](#toc0_)


Consider a worker choosing hours of labor, $L\in[0,24]$, to maximize utility: 

$$
\begin{align*}
V(w,\tau,G)&=\max_{L\in[0,24]}\ln\left(C^{\alpha}G^{1-\alpha}\right)-\nu\frac{L^{2}}{2}\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}
$$

where 

* $C$ is *private* consumption with weight $\alpha\in(0,1)$.
* $\kappa > 0$ is the *free private* consumption component.
* $C = (1-\tau)wL$ is the *costly private* consumption component.
* $w > 0 $ is the real wage.
* $\tau \in (0,1)$ is the labor-income tax rate.
* $G > 0 $ is *government* consumption with weight $1-\alpha$.
* $\nu > 0$ is the disutility of labor scaling factor


The baseline parameters are:

$$
\begin{align*}
\alpha &= 0.5\\
\kappa &= 1.0\\
\nu &= \frac{1}{2\cdot16^2} \\
w &= 1.0 \\ 
\tau &= 0.30 \\
\end{align*}
$$

**Question 1:** Verify that the optimal labor supply choice is $L^{\star}(\tilde{w}) =\frac{-\kappa+\sqrt{\kappa^{2}+4\frac{\alpha}{\nu}\tilde{w}^2}}{2\tilde{w}}$, where $\tilde{w} = (1-\tau)w$, for $G\in\left\{1.0 , 2.0\right\}$.

In [7]:
import Exam
from Exam import UtilityPlotter

kappa = 1.0
alpha = 0.5
nu = 1 / (2 * 16**2)
w = 1.0
tau = 0.30

utility_plotter = UtilityPlotter(kappa, alpha, nu, tau, w)
utility_plotter.plot_utility()

ImportError: cannot import name 'UtilityPlotter' from 'Exam' (/Users/mathildemaix/projects-2023-n-m-2/exam/Exam.py)

In [ ]:
# Defining the parameter symbols
L, w_tilde, kappa, alpha, nu, C, G, tau = sm.symbols('L w_tilde kappa alpha nu C G tau')

# Defining utility & consumption constraint (we have already substituted the expression for w_tilde in the consumption constraint)
utility = sm.log(C**alpha * G**(1-alpha)) - (nu * (L**2 / 2))
consumption_constraint = sm.Eq(C, kappa + w_tilde * L)

consumption_constraint_solved = sm.solve(consumption_constraint, C)

# Inserting expression for C in the utility function
utility_subs = utility.subs(C, consumption_constraint_solved[0]) 

# Different values of G to test
G_values = [1.0, 2.0]

# Calculate the optimal L for each G value using list comprehension
optimal_L = [sm.solve(sm.Eq(sm.diff(utility_subs.subs(G, G_val), L), 0), L)[0] for G_val in G_values]
optimal_L[0]

 

In [ ]:
# saving the function
optimal_L_func = sm.lambdify(args=(kappa,nu,alpha,w_tilde),expr=optimal_L[0])

We have verified that $L^{\star}(\tilde{w}) =\frac{-\kappa+\sqrt{\kappa^{2}+4\frac{\alpha}{\nu}\tilde{w}^2}}{2\tilde{w}}$ where $\tilde{w} = (1-\tau)w$ is the optimal labor supply  choice for $G\in\left\{1.0 , 2.0\right\}$. Our expression is not exactly identically expressed, but it is the same function, as 

$$\frac{-\kappa+\sqrt{\kappa^{2}+4\frac{\alpha}{\nu}\tilde{w}^2}}{2\tilde{w}} =\frac{0.5(-\kappa\nu-\sqrt{\nu(4\alpha \tilde{w}^{2}+\kappa^{2}}\nu)}{\tilde{w}\nu}$$

**Question 2:** Illustrate how $L^{\star}(\tilde{w})$ depends on $w$.

To illustrate how $L^{\star}(\tilde{w})$ depends on $w$ we plot the optimal labor supply choice in a graph. 

In [ ]:
w_tilde_values = np.linspace(0.01, 2, 100)  
kappa = 1.0  
alpha = 0.5  
nu = 1 / (2 * 16**2) 
w = 1.0
tau = 0.30

w_tilde = (1- tau)* w
w_tilde_values = np.linspace(0.01, 2, 100)
L_values = optimal_L_func(kappa, nu, alpha, w_tilde_values)  # Replace with actual parameter values

# Plot the graf
plt.plot(w_tilde_values, L_values)
plt.xlabel('w_tilde')
plt.ylabel('L^star')
plt.title('Plot of L^star(w_tilde)')
plt.grid(True)
plt.show()


In [ ]:
def L_star(kappa, alpha, nu, tau, w):
    w_tilde = (1 - tau) * w
    return (-kappa + np.sqrt(kappa**2 + 4 * alpha / nu * w_tilde**2)) / (2 * w_tilde)

kappa = 1.0  
alpha = 0.5  
nu = 1 / (2 * 16**2) 
w = 1.0
tau = 0.30

w = np.linspace(-10, 10, 100)  # Generate an array of w values
L = L_star(kappa, alpha, nu, tau, w)

plt.plot(w, L)
plt.xlabel('w_tilde')
plt.ylabel('L^star')
plt.title('Plot of L^star(w_tilde)')
plt.xlim(0, max(w))
plt.ylim(0, max(L))
plt.grid(True)
plt.show()

The variable $\tilde{w} represents the after-tax real wage and is used in the utility maximization problem to account for the influence of taxation on the worker's decision-making. By incorporating the after-tax real wage, the model considers the trade-off between work and leisure, which allows us to analysise how taxation influences labor supply choices.

The graph of $L^*$ as a function of $\tilde{w}$ illustrates the relationship between the optimal labor supply choice and the after-tax real wage. It lets us understand how changes in the after-tax real wage affect the worker's decision to supply labor. The graph converges towards a point where further increases in the after-tax real wage has diminishing effects on the optimal labor supply choice. At this saturation point, workers may have satisfied their level of consumption and will prefer to use more time on leisure activities rather than labor.


We now consider a government, who chooses $\tau$ and spend all of the taxes on government consumption so:

$$
G = \tau w L^{\star}((1-\tau)w)
$$

**Question 3:** Plot the implied $L$, $G$ and worker utility for a grid of $\tau$-values.


In [ ]:
# Defining the functions
def L_star(kappa, alpha, nu, w_tilde):
    return (-kappa + np.sqrt(kappa**2 + 4 * alpha / nu * w_tilde**2)) / (2 * w_tilde)

def G(kappa, alpha, nu, tau, w):
    w_tilde = (1 - tau) * w
    return tau * w * L_star(kappa, alpha, nu, w_tilde)*((1- tau)*w)

def L_starG(kappa, alpha, nu, tau, w):
    return G(kappa, alpha, nu, tau, w) / (tau*(1-tau)*w**2)

def V(kappa, alpha, nu, tau, w):
    L = np.linspace(0, 24, 100)  # Grid of L values
    C = kappa + (1 - tau) * w * L
    utility = np.log(C**alpha * G(kappa, alpha, nu, tau, w)**(1 - alpha)) - nu * L**2 / 2
    return np.max(utility)

# Parameters
kappa = 1.0  
alpha = 0.5  
nu = 1 / (2 * 16**2) 
w = 1.0
tau = 0.30 

tau_values = np.linspace(0, 1, 100)  # Grid of tau values

L_values = L_starG(kappa, alpha, nu, tau_values, w)
G_values = G(kappa, alpha, nu, tau_values, w)
V_values = np.zeros_like(tau_values)

for i, tau in enumerate(tau_values):
    V_values[i] = V(kappa, alpha, nu, tau, w)

plt.figure(figsize=(10, 6))
plt.subplot(2, 2, 1)
plt.plot(tau_values, L_values)
plt.xlabel('tau')
plt.ylabel('L^star')
plt.title('Plot of L^star')

plt.subplot(2, 2, 2)
plt.plot(tau_values, G_values)
plt.xlabel('tau')
plt.ylabel('G')
plt.title('Plot of G')

plt.subplot(2, 2, 3)
plt.plot(tau_values, V_values)
plt.xlabel('tau')
plt.ylabel('V')
plt.title('Plot of V')

plt.tight_layout()
plt.show()



TEXT TIL GRAFER

**Question 4:** Find the socially optimal tax rate $\tau^{\star}\in(0,1)$ maximizing worker utility. Illustrate your result.

In [ ]:
def V(tau, w, kappa, alpha, nu):
    w_tilde = (1 - tau) * w
    L_star = (-kappa + np.sqrt(kappa**2 + 4 * alpha/nu * w_tilde**2)) / (2 * w_tilde)
    C = kappa + (1 - tau) * w * L_star
    G = tau * w * L_star*((1-tau)*w)
    return np.log(C**alpha * G**(1 - alpha)) - nu * L_star**2 / 2

def maximize_V(w, kappa, alpha, nu):
    result = minimize_scalar(lambda tau: -V(tau, w, kappa, alpha, nu), bounds=(0, 1), method='bounded')
    return result.x

kappa = 1.0  
alpha = 0.5  
nu = 1 / (2 * 16**2) 
w = 1.0

max_tau = maximize_V(w, kappa, alpha, nu)
max_V = V(max_tau, w, kappa, alpha, nu)

print("Optimal tau:", max_tau)
print("Maximum V:", max_V)

# Generate tau values for plotting
tau_vals = np.linspace(0, 1, 100)

# Calculate V values
V_vals = [V(tau, w, kappa, alpha, nu) for tau in tau_vals]

# Plot V(tau)
plt.plot(tau_vals, V_vals)
plt.scatter(max_tau, max_V, color='red', label='Max V')
plt.xlabel('tau')
plt.ylabel('V')
plt.legend()
plt.title('Objective Function V(tau)')
plt.show()

A more general preference formulation for the worker is:

$$
\begin{align*}
\mathcal{V}(w,\tau,G)&=\max_{L\in[0,24]}\frac{\left[ \left( \alpha C^{\frac{\sigma-1}{\sigma}}+(1-\alpha) G^{\frac{\sigma-1}{\sigma}} \right)^{\frac{\sigma}{\sigma-1} }\right]^{1-\rho}-1}{1-\rho}- \nu\frac{L^{1+\varepsilon}}{1+\varepsilon},\,\,\,\varepsilon,\rho,\sigma>0,\,\,\,\rho,\sigma\neq1\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}    
$$

Optimal labor supply is now $L^{\star}(\tilde{w},G)$.

Questions 5 and 6 must be answered with the general formulation, and for 2 different set of parameters:

- Set 1:  $\sigma = 1.001$, $\rho = 1.001$ and $\varepsilon = 1.0$.
- Set 2:  $\sigma = 1.5$, $\rho = 1.5$ and $\varepsilon = 1.0 $.

**Question 5:** Find the $G$ that solves $G = \tau w L^{\star}((1-\tau)w,G)$ using the $\tau$ found in question 4.

*Hint: First write code that solves the worker problem for given values of $G$ and $\tau$. Then find the correct G based on this.*

In [ ]:
import scipy.optimize as optimize

def utility(L, w, tau, G, alpha, kappa, sigma, rho, nu, epsilon):
    C = kappa + (1 - tau) * w * L
    return ((((alpha * C**((sigma-1)/sigma) + (1 - alpha) * G**((sigma-1)/sigma))**(sigma/(sigma-1)))**(1-rho)-1)/(1- rho)) - nu * (L**(1+epsilon))/(1+epsilon)

def objective(G, *args):
    w, tau, alpha, kappa, sigma, rho, nu, epsilon = args
    L_opt = optimize.fminbound(lambda L: -utility(L, w, tau, G, alpha, kappa, sigma, rho, nu, epsilon), 0, 24)
    return G - tau * w * L_opt

# Parameter values for Set 1
w =  1.0
tau = 0.33817131261325806  # Given tau value
alpha =  0.5
kappa =  1.0
nu = 1 / (2 * 16**2)

# Set 1: sigma = 1.001, rho = 1.001, epsilon = 1.0
sigma_1 = 1.001
rho_1 = 1.001
epsilon_1 = 1.0  # Set the value of epsilon for Set 1

# Set 2: sigma = 1.5, rho = 1.5, epsilon = 1.0
sigma_2 = 1.5
rho_2 = 1.5
epsilon_2 = 1.0  # Set the value of epsilon for Set 2

# Solve for G for Set 1
initial_guess = 1.0  # Initial guess for G in Set 1
args_1 = (w, tau, alpha, kappa, sigma_1, rho_1, nu, epsilon_1)
result_1 = optimize.root(objective, initial_guess, args=args_1)
G_solution_1 = result_1.x[0]
L_opt_1 = optimize.fminbound(lambda L: -utility(L, w, tau, G_solution_1, alpha, kappa, sigma_1, rho_1, nu, epsilon_1), 0, 24)

# Solve for G for Set 2
initial_guess = 1.0  # Initial guess for G in Set 2
args_2 = (w, tau, alpha, kappa, sigma_2, rho_2, nu, epsilon_2)
result_2 = optimize.root(objective, initial_guess, args=args_2)
G_solution_2 = result_2.x[0]
L_opt_2 = optimize.fminbound(lambda L: -utility(L, w, tau, G_solution_2, alpha, kappa, sigma_2, rho_2, nu, epsilon_2), 0, 24)

# Print results
print("Set 1:")
print("Optimal labor supply (L^*):", L_opt_1)
print("Corresponding value of G:", G_solution_1)
print()
print("Set 2:")
print("Optimal labor supply (L^*):", L_opt_2)
print("Corresponding value of G:", G_solution_2)


**Question 6:** Find the socially optimal tax rate, $\tau^{\star}$, maximizing worker utility, while keeping $G = \tau w L^{\star}((1-\tau)w,G)$.

from scipy.optimize import minimize
def utility6(tau, L, w, G, alpha, kappa, sigma, rho, nu, epsilon):
    C = kappa + (1 - tau) * w * L
    return ((((alpha * C**((sigma-1)/sigma) + (1 - alpha) * G**((sigma-1)/sigma))**(sigma/(sigma-1)))**(1-rho)-1)/(1- rho)) - nu * (L**(1+epsilon))/(1+epsilon)

def constraint6(tau, L, w, G):
    return G - tau * w * L

def objective6(tau, *args):
    L, w, G, alpha, kappa, sigma, rho, nu, epsilon = args
    return -utility6(tau, L, w, G, alpha, kappa, sigma, rho, nu, epsilon)  # Minimize negative utility

def find_optimal_tau(L, w, G, alpha, kappa, sigma, rho, nu, epsilon):
    initial_guess = 0.5  # Initial guess for tau
    result = minimize(objective6, initial_guess, args=(L, w, G, alpha, kappa, sigma, rho, nu, epsilon), constraints={'type': 'eq', 'fun': constraint6, 'args': (L, w, G)})
    optimal_tau = result.x[0]
    return optimal_tau

# Parameter values for Set 1
w =  1.0
tau = 0.33817131261325806  # Given tau value
alpha =  0.5
kappa =  1.0
nu = 1 / (2 * 16**2)

# Set 1: sigma = 1.001, rho = 1.001, epsilon = 1.0
sigma_1 = 1.001
rho_1 = 1.001
epsilon_1 = 1.0  # Set the value of epsilon for Set 1

optimal_tau = find_optimal_tau(L, w, G, alpha, kappa, sigma_1, rho_1, nu, epsilon_1)
print("Optimal tau:", optimal_tau)

## 2. <a id='toc2_'></a>[Problem 2: Labor adjustment costs](#toc0_)

You own a hair salon. You employ hairdressers, $\ell_t$, to produce haircuts, $y_t = \ell_t$.

The wage for each haridresser is $w$.

The demand for haircuts implies that the price of haircuts you can charge is $p_t = \kappa_t y_t^{-\eta}$, where $\kappa_t$ is a demand-shock and $\eta \in (0,1)$ measures the elasticity of demand.

Profits are:

$$
\Pi_t = p_t y_t - w \ell_t = \kappa_t \ell_t^{1-\eta} - w \ell_t
$$

Baseline parameters are:
- $\eta = 0.5$
- $w = 1.0$

**Question 1:** Verify numerically that $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ maximises profits, for $\kappa\in\left\{1.0 , 2.0\right\}$.

In [ ]:
p_t = sm.symbols('p_t')
y_t = sm.symbols('y_t')
w = sm.symbols('w')
kappa_t = sm.symbols('kappa_t')
ell_t = sm.symbols('ell_t')
eta = sm.symbols('eta')

# Define the profit function
profit_function = kappa_t * ell_t**(1 - eta) - w * ell_t

# Calculate the derivative of the profit function with respect to ell
derivative = sm.diff(profit_function, ell_t)

# Set the derivative equal to zero and solve for ell
optimal_ell = sm.solve(derivative, ell_t)

# Print the optimal ell expression
print("Optimal ell expression:")
print(optimal_ell)

In [ ]:
eta = 0.5
w = 1.0
kappa_values = [1.0, 2.0]

for kappa in kappa_values:
    # Calculate ell_t
    ell_t = (kappa * (1 - eta) / w) ** (1 / eta)

    # Calculate profits
    profits = kappa * ell_t ** (1 - eta) - w * ell_t

    # Print the results
    print(f"For kappa = {kappa:.1f}, the maximum profit is = {profits:.2f}")


We now consider a *dynamic* version of the model.

* The demand-shock is a so-called AR(1) in logs, 

$$
\log \kappa_{t} = \rho \log \kappa_{t-1} + \epsilon_{t},\,\,\, \epsilon_{t+1} \sim \mathcal{N}(-0.5\sigma_{\epsilon}^2,\sigma_{\epsilon})
$$

* Any hiring or firing implies a fixed adjustment cost, $\iota > 0 $.
* Future profits are discounted with a monthly factor of $R \in (0,1)$.

The initial demand shock is $\kappa_{-1} = 1$ and the planning horizon is 10 years, i.e. 120 months so $t \in \{0,1,2,\dots,119\}$. Initially you don't have any employees, $\ell_{-1}=0$


The *ex post* value of the salon is *conditional* on the shock series is:

$$
h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119}) = \left[\sum_{t=0}^{119}R^{-t}\left[\kappa_{t}\ell_{t}^{1-\eta}-w\ell_{t}-\boldsymbol{1}_{\ell_{t}\neq\ell_{t-1}}\iota\right]\right]
$$

The *ex ante* expected value of the salon can be approximated by

$$
H = \mathbb{E}[h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119})] \approx \frac{1}{K}\sum_{k=0}^{K} h(\epsilon_0^k,\epsilon_1^k,\dots,\epsilon_{119}^k)
$$

where each $k\in\{0,1,\dots,K-1\}$ is a random shock series. Maximizing profitability means maximizing $H$.


Baseline parameters are: 

- $\rho = 0.90$
- $\iota = 0.01$
- $\sigma_{\epsilon} = 0.10$
- $R = \left(1+0.01\right)^{1/12}$

**Question 2:** Calculate $H$ if the policy  $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ from question 1 is followed. Choose $K$ so the approximation is good enough to not affect your results substantially.

In [ ]:
import numpy as np

eta = 0.5
w = 1.0
rho = 0.90
iota = 0.01
sigma_epsilon = 0.10
R = (1 + 0.01) ** (1 / 12)
K = 1000  # Number of shock series

# Simulate shock series
np.random.seed(1)
epsilon_series = np.random.normal(-0.5 * sigma_epsilon**2, sigma_epsilon, size=(K, 120))

# Calculate value function for each shock series
value_functions = []
for k in range(K):
    kappa_series = np.exp(np.zeros(120))
    ell_series = np.zeros(120)
    value_function = 0

    for t in range(120):
        kappa_series[t] = np.exp(rho * np.log(kappa_series[t - 1]) + epsilon_series[k, t])
        ell_series[t] = (1 - eta) * kappa_series[t] / w ** (1 / eta)

        if t > 0 and ell_series[t] != ell_series[t - 1]:
            value_function += R ** (-t) * (kappa_series[t] * ell_series[t] ** (1 - eta) - w * ell_series[t] - iota)

    value_functions.append(value_function)

# Calculate expected value of the salon
H = np.mean(value_functions)

print(f"Expected value of the salon (H): {H}")


We have chosen K=1000. Higher values of K provide more precise estimations of the expected value, but also increases the loading time in visual code. 

Next, we consider policies on the form:

$$

\ell_{t}=\begin{cases}
\ell_t^{\ast}  & \text{if }\left|\ell_{t-1}-\ell_t^{\ast} \right|>\Delta\\
\ell_{t-1} & \text{else }
\end{cases}
\\
\text{where}\,\,\ell_t^{\ast} = \left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}} \\

$$
With $\Delta \geq 0$ and $\Delta = 0$ being the previous policy.



**Question 3:** Calculate $H$ if the policy above was followed with $\Delta = 0.05$. Does it improve profitability?

In [ ]:
eta = 0.5
w = 1.0
rho = 0.90
iota = 0.01
sigma_epsilon = 0.10
R = (1 + 0.01) ** (1 / 12)
K = 1000  # Number of shock series
delta = 0.05  # Adjustment threshold

# Simulate shock series
np.random.seed(1)
epsilon_series = np.random.normal(-0.5 * sigma_epsilon**2, sigma_epsilon, size=(K, 120))

# Calculate value function for each shock series
value_functions = []
for k in range(K):
    kappa_series = np.exp(np.zeros(120))
    ell_series = np.zeros(120)
    value_function = 0

    for t in range(120):
        kappa_series[t] = np.exp(rho * np.log(kappa_series[t - 1]) + epsilon_series[k, t])

        ell_star = (1 - eta) * kappa_series[t] / w ** (1 / eta)

        if t == 0 or abs(ell_series[t - 1] - ell_star) > delta:
            ell_series[t] = ell_star
        else:
            ell_series[t] = ell_series[t - 1]

        if t > 0 and ell_series[t] != ell_series[t - 1]:
            value_function += R ** (-t) * (kappa_series[t] * ell_series[t] ** (1 - eta) - w * ell_series[t] - iota)

    value_functions.append(value_function)

# Calculate expected value of the salon
H = np.mean(value_functions)

print(f"New expected value of the salon (H): {H}")


The value of $H$ obtained with the updated policy is lower than the value obtained with the previous policy, it suggests that the new policy with $\Delta = 0.05$ does not improve profitability compared to the previous policy.



**Question 4:** Find the optimal $\Delta$ maximizing $H$. Illustrate your result.

In [ ]:
eta = 0.5
w = 1.0
rho = 0.90
iota = 0.01
sigma_epsilon = 0.10
R = (1 + 0.01) ** (1 / 12)
K = 1000  # Number of shock series

delta_values = np.linspace(0, 0.1, 100)  # Range of delta values to search over
H_values = []

# Simulate shock series
np.random.seed(1)
epsilon_series = np.random.normal(-0.5 * sigma_epsilon ** 2, sigma_epsilon, size=(K, 120))

# Calculate value function for each delta value
for delta in delta_values:
    value_functions = []
    for k in range(K):
        kappa_series = np.exp(np.zeros(120))
        ell_series = np.zeros(120)
        value_function = 0

        for t in range(120):
            kappa_series[t] = np.exp(rho * np.log(kappa_series[t - 1]) + epsilon_series[k, t])

            ell_star = (1 - eta) * kappa_series[t] / w ** (1 / eta)

            if t == 0 or abs(ell_series[t - 1] - ell_star) > delta:
                ell_series[t] = ell_star
            else:
                ell_series[t] = ell_series[t - 1]

            if t > 0 and ell_series[t] != ell_series[t - 1]:
                value_function += R ** (-t) * (
                        kappa_series[t] * ell_series[t] ** (1 - eta) - w * ell_series[t] - iota)

        value_functions.append(value_function)

    # Calculate expected value of the salon for the current delta value
    H = np.mean(value_functions)
    H_values.append(H)

# Find the optimal delta that maximizes H
optimal_delta = delta_values[np.argmax(H_values)]
max_H = np.max(H_values)

# Plot the results
plt.plot(delta_values, H_values)
plt.xlabel('Delta')
plt.ylabel('Expected value of the salon (H)')
plt.title('Optimization of Delta')
plt.axvline(x=optimal_delta, color='r', linestyle='--', label='Optimal Delta')
plt.legend()
plt.show()

print(f"Optimal Delta: {optimal_delta}")
print(f"Maximum H value: {max_H}")



**Question 5:** Suggest an alternative policy you believe might improve profitability. Implement and test your policy.



In [ ]:
# write your code here

## 3. <a id='toc3_'></a>[Problem 3: Global optimizer with refined multi-start](#toc0_)

We consider the Griewank function:

$$ f(\boldsymbol{x}) = \sum^n_{i=1} \frac{x^2_i}{4000}-\prod^n_{i=1}\cos\left(\frac{x_i}{\sqrt{i}}\right)+1$$

The **global minimum** of this function is $f(0,0) = 0$ (remember: $\cos(0)=1$).<br>
But the function also have a lot of **local minima**.

A **refined global optimizer with multi-start** is:

1. Choose *bounds* for $\mathbf{x}$ and *tolerance* $\tau > 0$.
2. Choose number of *warm-up iterations*, $\underline{K} > 0$ and *maximum number of iterations*, $K > \underline{K}$.
3. In each iteration for $k \in \{0,1,\dots,K-1\}$:

    A. Draw random $\mathbf{x}^k$ uniformly within chosen bounds.

    B. If $k < \underline{K}$ go to step E.

    C. Calculate $\chi^k = 0.50\cdot\frac{2}{1+\exp((k-\underline{K})/100)}$  

    D. Set $\mathbf{x}^{k0} = \chi^k \mathbf{x}^k + (1-\chi^k)\mathbf{x}^{\ast} $

    E. Run optimizer with $\mathbf{x}^{k0}$ as initial guess and $\mathbf{x}^{k\ast}$ as result.

    F. Set $\mathbf{x}^{\ast} = \mathbf{x}^{k\ast}$ if $k = 0$ or $f(\mathbf{x}^{k\ast}) < f(\mathbf{x}^{\ast})$

    G. If $f(\mathbf{x}^{\ast}) < \tau$ go to step 4.

4. Return the result $\mathbf{x}^{\ast}$.

As settings we choose:

* $x_1,x_2 \in  [-600,600]$
* $\tau = 10^{-8}$
* $\underline{K}=10$
* $K=1000$

The optimizer in Step 3.E is `BFGS` with a tolerance of $\tau$.

**Question 1:** Implement the refined global optimizer with multi-start. Illustrate how the effective initial guesses $\mathbf{x}^{k0}$ vary with the iteration counter $k$.

In [ ]:
from Exam import RefinedGlobalOptimizer

# Create an instance of the RefinedGlobalOptimizer class with warm-up iterations = 10
optimizer_10 = RefinedGlobalOptimizer(warm_up_iterations=10)
optimizer_10.optimize()
optimizer_10.print_convergence_iteration()
optimizer_10.plot_effective_initial_guesses()

We can see that the optimizers converge all over the place, and end up in local minima instead of the global minimum. 


**Question 2:** Is it a better idea to set $\underline{K} = 100$? Is the convergence faster?

In [ ]:
# Create an instance of the RefinedGlobalOptimizer class with warm-up iterations = 10
optimizer_100 = RefinedGlobalOptimizer(warm_up_iterations=100)
optimizer_100.optimize()
optimizer_100.print_convergence_iteration()
optimizer_100.plot_effective_initial_guesses()